## Criptosistema de McEliece con códigos BCH
#### Autor: Javier Ojeda Baena
#### DNI: 75943022N

In [1]:
from sage.coding.bch_code import BCHCode
from sage.coding.cyclic_code import CyclicCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder
from IPython.display import HTML

In [2]:
def random_word(n, F):
    """
    Genera una palabra aleatoria de longitud n utilizando los elementos de un cuerpo finito F.

    Parámetros:
        - n : longitud de la palabra que se quiere generar.
        - F : cuerpo finito del que se van a coger los elementos para generar la palabra aleatoria.

    Devuelve:
        Palabra aleatoria de longitud n de elementos de F
    """
    word = []

    while len(word) != n:
        word = word + [choice(F.list())]

    word = vector(word, F)
    
    return word

In [3]:
def random_error(n, n_errors, F):
    """
    Genera un vector de errores aleatorios de longitud n, que tiene como máximo n errors 
    elementos no nulos, utilizando elementos del cuerpo finito F.

    Parámetros:
        - n : longitud del vector de errores que se quiere generar.
        - n_errors : número máximo de elementos no nulos que se quieren en el vector de errores.
        - F : cuerpo finito del que se van a coger los elementos para generar el vector de errores aleatorios.
        
    Devuelve:
        - Vector de errores aleatorios con n elementos, n_errors elementos no nulos de elementos de F
    """
    e = [0] * n
    
    i = 0
    
    while i < n_errors:
        p = randint(0, n-1)
        e[p] = choice(F.list())
        i += 1

    e = vector(e, F)
    
    return e

In [4]:
class BCHEncoder(Encoder):
    """
    Clase que representa un codificador BCH.
    """
    
    
    def __init__(self, code):
        """
        Constructor de la clase.

        Parámetros:
            - code: código BCH.
        """
        super(BCHEncoder, self).__init__(code)
        
        
    def _repr_(self):
        """
        Método que devuelve una representación en cadena de caracteres de la clase.

        Devuelve:
            - Cadena de caracteres que representa el objeto.
        """
        return "Encoder for {}".format(self.code())
    
    
    def encode (self, m):
        """
        Método que codifica un mensaje utilizando el código BCH.

        Parámetros:
            - m: mensaje a codificar.

        Devuelve:
            - Mensaje codificado con el código BCH.
        """
        return m * self.code().generator_matrix()

## Decodificador con el algoritmos de Sugiyama
El proceso final coincide con el algoritmo de Peterson-Gorenstein-Ziele, la única parte que cambia es la forma de calcular $\sigma(x)$ el polinomio localizador de errores:

1. Calculamos el síndrome $S(x)=\displaystyle\sum_{i=0}^{2t-1} S_{i+1}x^{i}$.


2. Sean $r_{-1}(x) = x^{2t}$, $r_0(x) = S(x)$, $b_{-1}(x) = 0$ y $b_0(x) = 1$. Repetimos los siguientes dos calculos encontrando $h_{i}(x),\,r_{i}(x),\,\text{ y }b_{i}(x)$ de forma inductiva para $i=1,\,2,\,\ldots,\,I$, hasta $I$ satisfaciendo $grad(r_{I-1}(x)\geq t$ y $grad(r_{I}(x))<t$:
$$r_{i-2}(x)=r_{i-1}(x)h_{i}+r_{i}(x),\text{ donde }grad(r_{i}(x))<grad(r_{i-1}(x))$$
$$b_{i}(x)=b_{i-2}(x)-h_{i}(x)b_{i-1}(x)$$


3. $\sigma(x)$ es algun escalar distinto de cero multiplicado por $b_{I}(x)$.

In [5]:
class BCHDecoderS(Decoder):
    """
    Implementación del decodificador con el algoritmo de Sugiyama para códigos BCH.
    """
        
    
    def __init__(self, code):
        """
        Constructor de la clase.

        Parámetros:
            - code: Código BCH.
        """
        super(BCHDecoderS, self).__init__(code, code.ambient_space(), "BCHDecoderS")
    
    
    def _repr_(self):
        """
        Método que devuelve una representación en cadena de caracteres de la clase.

        Devuelve:
            - Cadena de caracteres que representa el objeto.
        """
        return "Sugiyama decoder for {}".format(self.code())  

    
    
    def decode_to_code(self, y):
        """
        Dado un vector y calcula la palabra código más cercana a este con el algoritmo de Sugiyama

        Parámetros:
            - y: vector a decodificar.

        Devuelve:
            - La palabra código más cercana a y.
        """
        
        F = self.code().base_field()
        q = len(F.list())
        n = self.code().length()
        
        m = 1
        
        while (q**m%n != 1):
            m += 1
            
        L = GF(q^m)
        R.<x> = L[] 

        # Paso 1: calculamos S(x) e inicializamos los primeros valores del algoritmo
        
        t = floor((self.code().designed_distance()-1)/2)
        primitive_root = L.gen()**((q**m-1)//n)
        
        y_polynomial = 0
        for i in range(len(y)):
            y_polynomial += y[i]*x**i

            
        S = vector([y_polynomial(primitive_root**i) for i in range(1, 2*t+1)])
        
        S_polynomial = 0

        for j in range(len(S)):
            S_polynomial = S_polynomial+S[j]*x**j

        if S_polynomial == 0:
            return y
        
        i = 1
        
        f = x**(2*t)
        s = S_polynomial
        r = [f, s]
        b = [0, 1]
        
        # Paso 2: calculamos el el polinomio bI(x)

        while r[i].degree() >= t or r[i-1].degree() < t:
            (q, rs) = r[i-1].quo_rem(r[i])
            r.append(rs)
            b.append(b[i-1]-q*b[i])
            i += 1
        
        
        # Paso 3: el polinomio localizador de errores, sigma, es bI(x) por algún escalar distinto de 0
        
        sigma = b[i] / b[i].coefficients()[-1]
        
        # Paso 4: calculamos las posiciones de los errores, sabiendo que son los inversos de las raices
        #         de sigma(x)
        
        xi = [primitive_root**i for i in range(n)]
        
        k = []
        for root in sigma.roots():
            k.append(xi.index(root[0]**(-1)))
        
        # Paso 5: resolvemos el sistema formado por los sindromes, las posiciones de los errores, y las
        #         magnitudes de los mismos
        
        A = []
        
        for i in range(1, 2*t+1):
            equation = []
            for root in sigma.roots():
                equation.append(root[0]**(-i))
            A.append(equation)
            
        A = matrix(A)
        
        E = A.solve_right(S)
        
        # Calculamos el vector de errores
        
        e = [0] * n
        
        v = len(k)
        
        for i in range(v):
            e[k[i]] = E[i]
            
        y_decoded = y-vector(e, F)
        
        return y_decoded

    def decode_to_message(self, y):
        """
        Dado un vector y lo decodifica con el algoritmo de Sugiyama obteniendo el mensaje original.

        Parámetros:
            - y: vector a decodificar.

        Devuelve:
            - El mensaje decodificado.
        """
        c_decoded = self.decode_to_code(y)
        word = self.code().generator_matrix().solve_left(c_decoded)
        return word

## Criptosistema de McEliece

### Implementación

Generamos de forma aleatoria un $[n,\,k]$-código BCH binario, $C$, con un algoritmo de decodificación , $\mathcal{D}$, capaz de corregir hasta $t$ errores. A continuación obtenemos su matriz generadora $G$.\\
Una vez obtenido $G$, deberemos de obtener dos matrices aleatorias, $S$, matriz no singular de orden $k\times k$ y $P$ una matriz de permutaciones de orden $n\times n$. Con estas matrices calculamos $G'=SGP$. Esta matriz, es la matriz generadora de un código lineal con la misma tasa y distancia mínima que el código generado por $G$. $G'$ se denomina la matriz generadora pública.\\
La clave pública estará formada por $G'$ y $t$ (el número de errores que es capaz de corregir $\mathcal{D}$.
La clave privada estará formada por las matrices $G$, $S$ y $P$ y el algoritmo de decodificación asociado al código, $\mathcal{D}$ (por ejemplo en caso de usar códigos Reed-Solomon podemos usar como algoritmo de decodificación el de Sugiyama).

### Algoritmo de encriptación

Dada la información para encriptar dividida en bloques de $k$-bits. Si $u$ es uno de esos bloques, se manda el vector 
$$x=uG'+z$$
donde $G'$ es la matriz generadora pública, y $z$ es un vector random de error, de longitud $n$ y peso $t$.\\
Recivido el vector $x$, se puede recuperar $u$ de forma eficiente usando el siguiente algoritmo de desencriptación:

### Algoritmo de desincriptación

Calculamos $x'=xP^{-1}$ donde $P^{-1}$ es la inversa de la matriz de permutaciones $P$. Cabe recalcar que $uSG$ es una palabra código de $C$, luego:
$$x'=uSG+zP^{-1}.$$
$zP^{-1}$ es un vector de error con el mismo peso que $z$, es decir, $t$, luego al decodificar con el algoritmo $\mathcal{D}$, obtendremos $u'=uS$, el cual multiplicando por $S^{-1}$ por la derecha nos dará el mensaje original.

In [6]:
class McElieceEncrypter:

    """
    Implementación del sistema de encriptación de McEliece con código BCH.
    """
    
    def __init__(self, n, delta):
        """
        Constructor de la clase.

        Parámetros:
            - F: cuerpo finito que conformará la base de los elementos del código.
            - n: Longitud del código.
            - delta: distancia diseñada del código.
        """
        
        F = GF(2)
        
        C = BCHCode(F, n, delta, b = choice(range(1, n)))
        
        G = C.generator_matrix()
        
        k = G.nrows()

        S = matrix(F, k, [choice(F.list()) for i in range(k^2)])
        
        while rank(S) < k:
            i = randint(0, k-1)
            j = randint(0, k-1)
            S[i, j] = choice(F.list())

        columns = list(range(n))
        P = matrix(F, n)
        
        for i in range(n):
            l = randint(0, len(columns)-1)
            j = columns[l]
            P[i, j] = 1
            columns.remove(j)
            
        self._C = C
        self._S = S
        self._P = P
        self._G = G
        self._public_key = S * G * P
        
    def _repr_(self):
        """
        Método que devuelve una representación en cadena de caracteres de la clase.

        Devuelve:
            - Cadena de caracteres que representa el objeto.
        """
        return "McEliece Cryptosystem with {}".format(self._C)  
   
    
    def public_key(self):
        """
        Método que devuelve la clave pública.

        Devuelve:
            - La clave pública.
        """
        return self._public_key
    
    def encrypt(self, m):
        """
        Método para encriptar un mensaje.

        Parámetros:
            - m: mensaje a encriptar.
            
        Devuelve:
            - Mensaje encriptado.
        """
        e = random_error(self._public_key.ncols(), floor((self._C.designed_distance()-1)/2), self._C.base_field())
        c = m * self._public_key + e
        
        return c
    
    def decrypt(self, c):
        """
        Método para desencriptar un mensaje.

        Parámetros:
            - c: mensaje encriptado.
            
        Devuelve:
            - Mensaje desencriptado.
        """
        word = c * self._P^(-1)
        D = BCHDecoderS(self._C)
        word = D.decode_to_message(word)
        message = word * (self._S)^(-1)
        
        return message

## Ejemplo 1

### Generación del del encriptador y la palabra a codificar

Como es un código binario, utilizaremos $\mathbb{F}_{2}$ como el cuerpo base. La longitud será $7$ y la distancia diseñada $3$:

In [7]:
q = 2
F = GF(q)
n = 13
delta = 3
M = McElieceEncrypter(n, delta)
d = M.public_key().nrows()

print("Encriptador de McEliece:")
print(M._repr_())

print("\nClave pública:")
show(M.public_key())

Encriptador de McEliece:
McEliece Cryptosystem with [13, 1] BCH Code over GF(2) with designed distance 3

Clave pública:


/usr/lib/python3/dist-packages/sage/coding/cyclic_code.py:451: FutureWarning: This class/method/function is marked as experimental. It, its functionality or its interface might change without a formal deprecation.
See http://trac.sagemath.org/20284 for details.
  FE = RelativeFiniteFieldExtension(Fsplit, F,


[1 1 1 1 1 1 1 1 1 1 1 1 1]

Generamos una palabra aleatoria de tamaño $d=1$ de elementos del cuerpo finito, $\mathbb{F}_{2}$.

In [8]:
print("\nPalabra para codificar:")
message = random_word(d, F)
show(message)


Palabra para codificar:


(1)

Encriptamos la palabra:

In [9]:
print("\nPalabra codificada:")
c = M.encrypt(message)
show(c)


Palabra codificada:


(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

Desencriptamos para intentar recuperar el mensaje original:

In [10]:
print("\nMensaje desencriptado:")
message_decrypt = M.decrypt(c)
show(message_decrypt)


Mensaje desencriptado:


(1)

In [11]:
if message == message_decrypt:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje desencriptado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje desencriptado no coincide con el original</p>"

HTML(mensaje)

## Ejemplo 2

### Generación del del encriptador y la palabra a codificar

Como es un código binario, utilizaremos $\mathbb{F}_{2}$ como el cuerpo base. La longitud será $23$ y la distancia diseñada $5$:

In [12]:
q = 2
F = GF(q)
n = 23
delta = 5
M = McElieceEncrypter(n, delta)
d = M.public_key().nrows()

print("Encriptador de McEliece:")
print(M._repr_())

print("\nClave pública:")
show(M.public_key())

Encriptador de McEliece:
McEliece Cryptosystem with [23, 12] BCH Code over GF(2) with designed distance 5

Clave pública:


[0 0 1 0 1 0 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0]
[1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1]
[0 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 1]
[1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1]
[1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0]
[1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1]
[1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0]
[1 0 1 1 0 0 1 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 0]
[1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 0]
[1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1]
[0 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 0]
[0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 1]

Generamos una palabra aleatoria de tamaño $d=12$ de elementos del cuerpo finito, $\mathbb{F}_{2}$.

In [13]:
print("\nPalabra para codificar:")
message = random_word(d, F)
show(message)


Palabra para codificar:


(1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1)

Encriptamos la palabra:

In [14]:
print("\nPalabra codificada:")
c = M.encrypt(message)
show(c)


Palabra codificada:


(1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0)

Desencriptamos para intentar recuperar el mensaje original:

In [15]:
print("\nMensaje desencriptado:")
message_decrypt = M.decrypt(c)
show(message_decrypt)


Mensaje desencriptado:


(1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1)

In [16]:
if message == message_decrypt:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje desencriptado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje desencriptado no coincide con el original</p>"

HTML(mensaje)

## Ejemplo 3

### Generación del del encriptador y la palabra a codificar

Como es un código binario, utilizaremos $\mathbb{F}_{2}$ como el cuerpo base. La longitud será $49$ y la distancia diseñada $11$:

In [17]:
q = 2
F = GF(q)
n = 49
delta = 11
M = McElieceEncrypter(n, delta)
d = M.public_key().nrows()

print("Encriptador de McEliece:")
print(M._repr_())

print("\nClave pública:")
show(M.public_key())

Encriptador de McEliece:
McEliece Cryptosystem with [49, 4] BCH Code over GF(2) with designed distance 11

Clave pública:


[1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 1 1 0 0]
[1 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 0 0 0 1]
[1 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 0]
[0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 1 0]

Generamos una palabra aleatoria de tamaño $d=4$ de elementos del cuerpo finito, $\mathbb{F}_{2}$.

In [18]:
print("\nPalabra para codificar:")
message = random_word(d, F)
show(message)


Palabra para codificar:


(1, 1, 1, 0)

Encriptamos la palabra:

In [19]:
print("\nPalabra codificada:")
c = M.encrypt(message)
show(c)


Palabra codificada:


(1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1)

Desencriptamos para intentar recuperar el mensaje original:

In [20]:
print("\nMensaje desencriptado:")
message_decrypt = M.decrypt(c)
show(message_decrypt)


Mensaje desencriptado:


(1, 1, 1, 0)

In [21]:
if message == message_decrypt:
    mensaje = "<p style='color:green; font-size:16px;'></br>El mensaje desencriptado coincide con el original</p>"
else:
    mensaje = "<p style='color:red; font-size:16px;'></br>El mensaje desencriptado no coincide con el original</p>"

HTML(mensaje)